<a href="https://colab.research.google.com/github/towardsai/ai-tutor-rag-system/blob/main/notebooks/Structured(JSON)_PDF_Data_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q google-generativeai==0.5.4 openai tiktoken==0.7.0 llama-index-llms-gemini==0.1.10 arxiv

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.7/150.7 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 679.1/679.1 kB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.1/375.1 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 43.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.4 MB/s eta 0:00:00


In [ ]:
import os
Set the "OPENAI_API_KEY" in the Python environment. Will be used by OpenAI client later.
os.environ["OPENAI_API_KEY"] = "[OPENAI_API_KEY]"

# from google.colab import userdata
# os.environ["OPENAI_API_KEY"] = userdata.get('openai_api_key')

## 1. OpenAI Strucutred output (JSON)

In [ ]:
from openai import OpenAI
client = OpenAI()

In [ ]:
prompt = "Give me the names of the 10 best-selling books, their authors, the year they were published, and a concise summary in JSON format"

In [ ]:
# The response format- JSON schema
response_format_json = {
  "type": "json_schema",
  "json_schema": {
    "name": "Top10BestSellingBooks",
    "strict": True,
    "schema": {
      "type": "object",
      "properties": {
        "Top10BestSellingBooks": {
          "type": "array",
          "items": {
            "type": "object",
            "properties": {
              "title": { "type": "string" },
              "author": { "type": "string" },
              "yearPublished": { "type": "integer" },
              "summary": { "type": "string" }
            },

            "required": ["title", "author", "yearPublished", "summary"],
            "additionalProperties": False
          }
        }
      },
      "required": ["Top10BestSellingBooks"],
      "additionalProperties": False
    }
  }
}


In [ ]:
# Making the API call
response = client.chat.completions.create(
  model="gpt-4o-2024-08-06",
  response_format=response_format_json,
  messages=[
    {"role": "system", "content": "You are a helpful assistant designed to output JSON."},
    {"role": "user", "content": prompt}
  ]
)
print(response.choices[0].message.content)

{"Top10BestSellingBooks":[{"title":"Don Quixote","author":"Miguel de Cervantes","yearPublished":1605,"summary":"This Spanish novel follows the adventures of a nobleman obsessed with chivalric ideals, who sets out to revive knighthood under the name Don Quixote."},{"title":"A Tale of Two Cities","author":"Charles Dickens","yearPublished":1859,"summary":"Set in London and Paris before and during the French Revolution, this novel tells the story of Doctor Manette's release from imprisonment in the Bastille and his reunion with his daughter Lucie."},{"title":"The Lord of the Rings","author":"J.R.R. Tolkien","yearPublished":1954,"summary":"A high fantasy epic that follows hobbits, elves, and humans as they battle the dark lord Sauron, who wishes to conquer Middle-earth by using the One Ring."},{"title":"The Little Prince","author":"Antoine de Saint-Exupéry","yearPublished":1943,"summary":"A young boy who lives on a small asteroid recounts his life story and his travels to Earth, filled with

In [ ]:
print(type(response.choices[0].message.content))

<class 'str'>


In [ ]:
import json
result_book = json.loads(response.choices[0].message.content)
print(type(result_book))
print(result_book['Top10BestSellingBooks'][0])
print("-------------------------------------")
print(result_book['Top10BestSellingBooks'][0]['title'])

<class 'dict'>
{'title': 'Don Quixote', 'author': 'Miguel de Cervantes', 'yearPublished': 1605, 'summary': 'This Spanish novel follows the adventures of a nobleman obsessed with chivalric ideals, who sets out to revive knighthood under the name Don Quixote.'}
-------------------------------------
Don Quixote


## 2. Strucutred output from PDF + OpenAI + pdf2images

In [ ]:
!pip install -q -U pdf2image
!apt-get install poppler-utils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 49 not upgraded.
Need to get 186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.5 [186 kB]
Fetched 186 kB in 0s (704 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 123599 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.5_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.5) ...
Setting up poppler-utils (22.02.0-2ubuntu0.5) ...
Processing triggers for man-db (2.10.2-1) ...


In [ ]:
from huggingface_hub import hf_hub_download
file_path = hf_hub_download(repo_id="jaiganesan/ai_tutor_knowledge", filename="rag_research_paper.zip",repo_type="dataset",local_dir="/content")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:99: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


rag_research_paper.zip:   0%|          | 0.00/7.51M [00:00<?, ?B/s]

In [ ]:
!unzip /content/rag_research_paper.zip -d /content/

Archive:  /content/rag_research_paper.zip
   creating: /content/rag_research_paper/
  inflating: /content/rag_research_paper/2405.07437v2.pdf  
  inflating: /content/rag_research_paper/2407.01219v1.pdf  
  inflating: /content/rag_research_paper/2407.07858v1.pdf  
  inflating: /content/rag_research_paper/2407.08223v1.pdf  
  inflating: /content/rag_research_paper/2407.16833v1.pdf  
  inflating: /content/rag_research_paper/2407.21712v1.pdf  
  inflating: /content/rag_research_paper/2408.08067v2.pdf  
  inflating: /content/rag_research_paper/2408.08921v1.pdf  


In [ ]:
import os
from pdf2image import convert_from_path

pdf_directory = "/content/rag_research_paper"
output_dir = "/content/pages"
os.makedirs(output_dir, exist_ok=True)

pages_png = []

for pdf_file in os.listdir(pdf_directory):
    if pdf_file.endswith(".pdf"):
        pdf_path = os.path.join(pdf_directory, pdf_file)

        convert = convert_from_path(pdf_path, use_pdftocairo=True)

        pdf_output_dir = os.path.join(output_dir, os.path.splitext(pdf_file)[0])
        os.makedirs(pdf_output_dir, exist_ok=True)

        for page_num, image in enumerate(convert):
            page_filename = f"page-{str(page_num + 1).zfill(3)}.png"
            full_path = os.path.join(pdf_output_dir, page_filename)
            image.save(full_path)

            pages_png.append(full_path)

print(pages_png)

['/content/pages/2407.01219v1/page-001.png', '/content/pages/2407.01219v1/page-002.png', '/content/pages/2407.01219v1/page-003.png', '/content/pages/2407.01219v1/page-004.png', '/content/pages/2407.01219v1/page-005.png', '/content/pages/2407.01219v1/page-006.png', '/content/pages/2407.01219v1/page-007.png', '/content/pages/2407.01219v1/page-008.png', '/content/pages/2407.01219v1/page-009.png', '/content/pages/2407.01219v1/page-010.png', '/content/pages/2407.01219v1/page-011.png', '/content/pages/2407.01219v1/page-012.png', '/content/pages/2407.01219v1/page-013.png', '/content/pages/2407.01219v1/page-014.png', '/content/pages/2407.01219v1/page-015.png', '/content/pages/2407.01219v1/page-016.png', '/content/pages/2407.01219v1/page-017.png', '/content/pages/2407.01219v1/page-018.png', '/content/pages/2407.01219v1/page-019.png', '/content/pages/2407.01219v1/page-020.png', '/content/pages/2407.01219v1/page-021.png', '/content/pages/2407.01219v1/page-022.png', '/content/pages/2407.08223v1/pa

In [ ]:
from io import BytesIO
import base64
import json

# Function to encode the image
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

In [ ]:
# The response format- JSON schema
json_response_format = {
  "type": "json_schema",
  "json_schema": {
    "name": "research_paper_data",
    "strict": True,
    "schema": {
      "type": "object",
      "properties": {
        "research_paper_data": {
          "type": "array",
          "items": {
            "type": "object",
            "properties": {
                "name": { "type": "string" },
                "source": { "type": "string" },
                "content": { "type": "string"},

            },

            "required": ["name", "source", "content"],
            "additionalProperties": False
          }
        },
      },
      "required": ["research_paper_data"],
      "additionalProperties": False
    }
  }
}

system_instruction_prompt = """ You are an expert in Structured data extraction.
Task Description:
You are provided with an image that is a page from a research paper in PDF format. Your task is to extract all the relevant information from this
image. This includes all headlines, subheadlines, section headlines, and the content within those sections, as well as detailed information about
any figures, graphs, architectures, tables, and charts mentioned or displayed on the page. Include as much as content in one Json content output. Don't fragement the information.
Must consider note: Each generated response particluarly extracted content must have minimum of 100 words.
Extraction Requirements:
  1. (source) Research Paper ID: Identify and extract the research paper's Arxiv ID. This can often be found on the first page, typically on the left-hand side written from bottom to top.
    The format should be similar to an arXiv ID (e.g., "arXiv:2405.07437v2", "arXiv:2105.07327v1"). Important: Include the prefix "arXiv:" in the extracted
    ID (e.g., extract it as "arXiv:2105.07327v1", not "2105.07327v1").  **Please make sure one or more times the extracted Arxiv ID is correct or not.**

  2. (name) Headlines and Subheadlines: Extract all primary and secondary headlines, which may include sections like: Abstract, Introduction, Evaluation, Conclusion,
  References, etc. This also applies to subheadlines or subsection titles within those major sections. If the image lacks explicit headlines, create them based on the content.
  3. Content Extraction: Extract all relevant content under each headline, subheadline. Ensure the complete text under each section is captured,
  including paragraphs, lists, and bullet points. The content must also cover descriptions and information about any figures, graphs, architectures, tables,
  and charts present in the image.
    4.1. For each figure or graph, extract the following:
        - Caption or title (if provided).
        - Description of the figure, graph, or table.
        - Insights, trends, or comparisons described within the content.
        - Architecture designs or layouts shown, along with detailed analysis if available.

Handling Missing Headlines:
If the image does not include a visible headline, subheadline, or title, generate one based on the content. And Include all the content in that geadline.
 For example, if a section describes an evaluation method, create a headline like "Evaluation" and Include the mentioned content.

Figures, Graphs, and Tables: For any figures, graphs, architectures, tables, or charts present: Extract the title or caption. Take the main points, trends, or comparisons shown. Provide descriptions of
any architecture designs or layouts. Capture insights or observations related to the figure/chart that are mentioned in the text.

Important Notes:
- Fragmented Content: In cases where the text or content is fragmented (split across multiple parts), ensure the complete information is extracted and merged logically.
- Accuracy: Ensure that all extracted content is precise and complete, especially for any complex figures, tables, or architectures.
- No Summarization: Do not condense paragraphs or reduce the length of sentences. Extract each sentence and figure-related insight in full detail
without cutting or omitting any part of the content."""

In [ ]:
import arxiv
import re

def arxiv_extraction(arxiv_id):
  client = arxiv.Client()
  search = arxiv.Search(id_list=re.findall(r'(\d{4}\.\d{5}|\w+(?:-\w+)?/\d{7})', arxiv_id), max_results=1)
  results = client.results(search)

  for result in results:
    return result.title, result.pdf_url


In [ ]:
from openai import OpenAI
client = OpenAI()

desc = []
for page in pages_png:
  # Getting the base64
  base64_image = encode_image(page)

  try:
    system_prompt = {"role": "system","content":system_instruction_prompt}
    user_prompt = {"role": "user","content": [{"type": "text", "text": "Please extract the content from this research paper page image."},
                                              {"type": "image_url", "image_url": {"url":f"data:image/jpeg;base64,{base64_image}","detail": "high"}}]}

    response = client.chat.completions.create(
      model="gpt-4o-2024-08-06",
      response_format=json_response_format,
      messages= [system_prompt,user_prompt],)

    if response.choices[0].message.content is None:
      continue

    result = json.loads(response.choices[0].message.content)

    if 'page-001' in page:
      research_paper_id = result['research_paper_data'][0]['source']
      research_paper_title,research_paper_url = arxiv_extraction(research_paper_id)
      for i in range(len(result['research_paper_data'])):
        result['research_paper_data'][i]['source'] = research_paper_id
        result['research_paper_data'][i]['name'] = research_paper_title +":"+ result['research_paper_data'][i]['name']
        result['research_paper_data'][i]['url'] = research_paper_url

    if 'page-001' not in page:
      for i in range(len(result['research_paper_data'])):
        result['research_paper_data'][i]['source'] = research_paper_id
        result['research_paper_data'][i]['name'] = research_paper_title +":"+ result['research_paper_data'][i]['name']
        result['research_paper_data'][i]['url'] = research_paper_url

    desc.extend(result['research_paper_data'])

  except Exception as e:
    print(response.choices[0].finish_reason)
    print(f"Skipping {page}... error: {e}")
    continue


In [ ]:
len(desc)

359

In [ ]:
print("Content research paper title and Headline :",desc[6]['name'],"\n")
print("Content :",desc[6]['content'],"\n")
print("Source :",desc[6]['source'],"\n")
print("URL :",desc[6]['url'])

Content research paper title and Headline : Searching for Best Practices in Retrieval-Augmented Generation:Retriever and Generator Fine-tuning 

Content : Fine-tuning within the RAG framework is crucial for optimizing both retrievers and generators. Some research focuses on fine-tuning the generator to better utilize retriever context [30–32], ensuring faithful and robust generated content. Others fine-tune the retriever to learn to retrieve beneficial passages for the generator [33–35]. Holistic approaches treat RAG as an integrated system; fine-tuning both retriever and generator together to enhance overall performance [36–38], despite increased complexity and integration challenges. Several surveys have extensively discussed current RAG systems, covering aspects like text generation [7, 8], integration with LLMs [6, 39], multimodal [40], and AI-generated content [41]. While these surveys provide comprehensive overviews of existing RAG methodologies, selecting the appropriate approac

In [ ]:
source=[]
for i in range(len(desc)):
  source.append(desc[i]['source'])

source=list(set(source))
source

['Arxiv: 2407.08223',
 'Arxiv: 2407.16833',
 'Arxiv: 2408.08067',
 'Arxiv: 2407.21712',
 'Arxiv: 2407.01219',
 'Arxiv: 2408.08921',
 'Arxiv: 2407.07858',
 'Arxiv: 2405.07437']



---



In [ ]:
import json
with open("/content/ai_tutor_500.jsonl", "r") as file:
    result = [json.loads(line) for line in file]

result[1]

{'tokens': 3031,
 'doc_id': '60deb74f-d8b5-47a6-93f2-425887a46e33',
 'name': 'Named Entity Recognition in Ecommerce Industry  Custom model [Github Repo]  03/07/24',
 'url': 'https://towardsai.net/p/machine-learning/named-entity-recognition-in-ecommerce-industry-custom-model-github-repo-03-07-24',
 'source': 'tai_blog',
 'content': "Github Repo: https://github.com/vaibhawkhemka/ML-Umbrella/tree/main/NLP/Product-Categorization   From e-commerce to Customer support  all businesses require some kind of NER model to process huge amounts of texts from users.   To automate this whole  one requires NER models to extract relevant and important entities from text.   Final Result/OutputInput text = EL D68 (Green  32 GB) 3 GB RAM [3 GB RAM U+007C 32 GB ROM U+007C Expandable Upto 128 GB  15.46 cm (6.088 inch) Display  13MP Rear Camera U+007C 8MP Front Camera  4000 mAh Battery  Quad-Core Processor]   Output =   Green ->>>> COLOR 32 GB ->>>> STORAGE 3 GB RAM ->>>> RAM 3 GB RAM ->>>> RAM 32 GB ROM ->>

In [ ]:
result[1]['tokens']

3031

In [ ]:
import tiktoken
import uuid

def token_encoding_and_doc_id(content):
  encoding = tiktoken.get_encoding("cl100k_base")
  tokens = encoding.encode(content)

  num_tokens = len(tokens)
  doc_id = str(uuid.uuid5(uuid.NAMESPACE_DNS, content))

  return num_tokens, doc_id

for i in range(len(desc)):
  desc[i]['tokens'], desc[i]['doc_id'] = token_encoding_and_doc_id(desc[i]['content'])

for i in range(len(result)):
  result[i]['tokens'], result[i]['doc_id'] = token_encoding_and_doc_id(result[i]['content'])

In [ ]:
ai_tutor_knowledge = result + desc

In [ ]:
ai_tutor_knowledge[850]

{'name': 'Retrieval Augmented Generation or Long-Context LLMs? A Comprehensive Study and Hybrid Approach:References',
 'source': 'Arxiv: 2407.16833',
 'content': 'Yuri Kuratov, Aydar Bulatov, Petr Anokhin, Dmitry Sorokin, Artyom Sorsokin, and Mikhail Burtsev. 2024. In search of needles in a 10m haystack: Recurrent memory finds what lms miss. arXiv preprint arXiv:2402.10790.',
 'url': 'http://arxiv.org/pdf/2407.16833v1',
 'tokens': 74,
 'doc_id': '55194ca6-3fcc-50ba-914f-e07577bc9973'}

In [ ]:
ai_tutor_knowledge = [d for d in ai_tutor_knowledge if d['tokens'] > 100]

In [ ]:
with open('/content/ai_tutor_knowledge.jsonl', 'w') as f:
    for item in ai_tutor_knowledge:
        f.write(json.dumps(item) + '\n')

print("JSONL file saved successfully!")

JSONL file saved successfully!


In [ ]:
from google.colab import files
files.download('/content/ai_tutor_knowledge.jsonl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

------------